In [37]:
import os
import re
from openai import OpenAI
from dotenv import load_dotenv
from typing import TypedDict, List, Dict, Literal, Any, Callable
from serpapi import SerpApiClient

load_dotenv()

class TextBlock(TypedDict):
    type: Literal["text"]
    text: str

class Message(TypedDict):
    role: Literal["system", "user", "assistant", "tool"]
    content: List[TextBlock]   

In [49]:
class HelloAgenticsLLM:
    def __init__(self, model: str = None, apiKey: str = None, baseUrl: str = None, timeout: int = None):
        self.model = model or os.getenv('LLM_MODEL_ID')
        apiKey = apiKey or os.getenv('LLM_API_KEY')
        baseUrl = baseUrl or os.getenv('LLM_BASE_URL')
        timeout = timeout or int(os.getenv('LLM_TIMEOUT', 60))

        if not all([self.model, apiKey, baseUrl]):
            raise ValueError('模型ID，API密匙和服务地址必须提供')
        
        self.client = OpenAI(api_key=apiKey, base_url=baseUrl, timeout=timeout)

    def safe_stream_collect_text(self, response):
        collected = []

        for chunk in response:
            # --- 情况 1：chunk 是 tuple ---
            if isinstance(chunk, tuple):
                # 常见结构：(event_type, data)
                if len(chunk) < 2:
                    continue
                data = chunk[1]
            else:
                data = chunk

            # --- 情况 2：data 是 dict ---
            if isinstance(data, dict):
                choices = data.get("choices")
                if not choices:
                    continue

                delta = choices[0].get("delta", {})
                text = delta.get("content")

            # --- 情况 3：OpenAI SDK 对象 ---
            else:
                if not hasattr(data, "choices"):
                    continue
                if not data.choices:
                    continue

                choice = data.choices[0]
                if not hasattr(choice, "delta"):
                    continue

                text = getattr(choice.delta, "content", None)

            # --- 最终兜底 ---
            if not isinstance(text, str):
                continue

            # 过滤 think 标签
            if "<think>" in text or "</think>" in text:
                continue

            print(text, end="", flush=True)
            collected.append(text)

        return "".join(collected)

    def think(self, messages: List[Dict[str, str]], temperature: float = 0) -> str:
        print(f'🧠正字调用 {self.model} 模型...')
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=temperature,
                stream=True
            )

            print('✅ 大模型响应成功：')
            response_text = self.safe_stream_collect_text(response)

            return response_text

                
        except Exception as e:
            print(f"❌ 调用LLM API时发生错误: {e}")
            return None
llmClient = HelloAgenticsLLM()

In [13]:
input_data = [
    {"role": "system", "content": "You are a helpful assistant that writes Python code."},
    {"role": "user", "content": "写一个快速排序算法"}
]

In [14]:
print("\n\n ---调用LLM---")
responseText = llmClient.think(input_data)
if responseText:
    print("\n\n ---完整模型响应---")
    print(responseText)



 ---调用LLM---
🧠正字调用 mimo-v2-flash-free 模型...
✅ 大模型响应成功：
好的，这是一个经典的快速排序算法实现，并附有详细的解释。

我们将使用最常见和易于理解的**原地（in-place）分区**方法。

### Python 代码实现

```python
def quick_sort(arr):
    """
    快速排序的主函数
    :param arr: 待排序的列表
    """
    # 辅助函数：调用真正的排序逻辑
    _quick_sort_helper(arr, 0, len(arr) - 1)
    return arr

def _quick_sort_helper(arr, low, high):
    """
    递归排序的辅助函数
    :param arr: 待排序的列表
    :param low: 子数组的起始索引
    :param high: 子数组的结束索引
    """
    # 基线条件：如果子数组只有一个或没有元素，则已经有序
    if low < high:
        # 1. 分区操作：获取基准元素（pivot）的最终位置
        pivot_index = partition(arr, low, high)

        # 2. 递归地对基准元素左边的子数组进行排序
        _quick_sort_helper(arr, low, pivot_index - 1)

        # 3. 递归地对基准元素右边的子数组进行排序
        _quick_sort_helper(arr, pivot_index + 1, high)

def partition(arr, low, high):
    """
    分区函数 (使用 Lomuto 分区方案)
    :param arr: 待排序的列表
    :param low: 起始索引
    :param high: 结束索引
    :return: 基准元素最终位置的索引
    """
    # 选择最后一个元素作为基准（pivot）
    pivot = arr[high]
    
    # i 指向小于基准的子数组的末尾

## ReAct
通过thought-action-observation方式

Thought (思考)： 这是智能体的“内心独白”。它会分析当前情况、分解任务、制定下一步计划，或者反思上一步的结果。
Action (行动)： 这是智能体决定采取的具体动作，通常是调用一个外部工具，例如 Search['华为最新款手机']。
Observation (观察)： 这是执行Action后从外部工具返回的结果，例如搜索结果的摘要或API的返回值。

## Tool
### Google网页查询工具SerpAPI


In [29]:
def search(query: str) -> str:
    print(f"🔍 正在执行 [SerpApi] 网页搜索: {query}")
    try:
        api_key = os.getenv('SERPAPI_API_KEY')
        if not api_key:
            return "错误:SERPAPI_API_KEY 未在 .env 文件中配置。"
        
        params = {
            'engine': "google",
            'q': query,
            "api_key": api_key,
            'gl': "cn",
            'hl': "zh-cn",
        }

        client = SerpApiClient(params)
        results = client.get_dict()

        # 智能解析:优先寻找最直接的答案
        if "answer_box_list" in results:
            return "\n".join(results["answer_box_list"])
        if "answer_box" in results and "answer" in results["answer_box"]:
            return results["answer_box"]["answer"]
        if "knowledge_graph" in results and "description" in results["knowledge_graph"]:
            return results["knowledge_graph"]["description"]
        if "organic_results" in results and results["organic_results"]:
            # 如果没有直接答案，则返回前三个有机结果的摘要
            snippets = [
                f"[{i+1}] {res.get('title', '')}\n{res.get('snippet', '')}"
                for i, res in enumerate(results["organic_results"][:3])
            ]
            return "\n\n".join(snippets)
        
        return f"对不起，没有找到关于 '{query}' 的信息。"

    except Exception as e:
        return f"搜索时发生错误: {e}"

### 构建通用的工具执行器

In [27]:
class ToolExecutor:
    """
    一个工具执行器，负责管理和执行工具。
    """
    def __init__(self):
        self.tools: Dict[str, Dict[str, Any]] = {}

    def registerTool(self, name:str, description:str, func: Callable):
        """
        向工具箱中注册一个新工具。
        """
        if name in self.tools:
            print(f"警告：工具 {name} 已存在，将被覆盖。")
        self.tools[name] = {"description": description, 'func':func}
        print(f"工具 {name} 已注册。")

    def getTool(self, name: str) -> Callable:
        """
        根据名称获取一个工具的执行函数。
        """

        return self.tools.get(name, {}).get('func')
    
    def getAvaliableTools(self) -> str:
        """
        获取所有可用工具的格式化描述字符串。
        """
        return "\n".join([
            f"- {name}: {info['description']}"
            for name, info in self.tools.items()
        ]) 

In [35]:
toolExecutor = ToolExecutor()

search_description = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
toolExecutor.registerTool(name="Search", description=search_description, func=search)

print("\n ---可用工具---")
print(toolExecutor.getAvaliableTools())

print("\n ---执行 Action: Search['英伟达最新的GPU型号是什么]---")
tool_name = "Search"
tool_input = "英伟达最新的GPU型号是什么"

tool_function = toolExecutor.getTool(tool_name)
if tool_function:
    observation = tool_function(tool_input)
    print("---观察 （Observation）---")
    print(observation)
else:
    print(f"错误：未找到名为'{tool_name}'的工具")

工具 Search 已注册。

 ---可用工具---
- Search: 一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。

 ---执行 Action: Search['英伟达最新的GPU型号是什么]---
🔍 正在执行 [SerpApi] 网页搜索: 英伟达最新的GPU型号是什么
---观察 （Observation）---
[1] 比较GeForce 系列最新一代显卡和前代显卡
比较最新一代RTX 30 系列显卡和前代的RTX 20 系列、GTX 10 和900 系列显卡。查看规格、功能、技术支持等内容。

[2] GeForce 显卡| NVIDIA
探索NVIDIA GeForce 显卡。RTX 50 系列和RTX 40 系列。

[3] 一文彻底读懂：英伟达GPU分类、架构演进和参数解析
Quadro系列是英伟达专业级GPU产品线，针对商业和专业应用领域进行了优化。常见的产品型号如NVIDIA RTX A6000、A5000等。 Quadro GPU具备强大的计算能力、大 ...


In [44]:
REACT_PROMPT_TEMPLATE = '''
请注意，你是一个有能力调用外部工具的智能助手。

可用工具如下：
{tools}

请严格按照以下的格式进行回应：

Thought：你的思考过程，用于分析问题，超姐任务和规划下一步行动
Action：你觉得采取的行动，必须是以下格式之一：
- '{{tool_name}}[{{tool_input}}]':调用一个可用工具。
- 'Finish[最终答案]':当你认为已经获得最终答案时。
- 当你收集到足够的信息，能够回答用户的最终问题是，你必须在Action:字段后使用 finish(answer='...')来输出最终答案。

现在，请开始解决以下问题：
Question: {question}
History: {history}
'''


In [52]:
class ReActAgent:
    def __init__(self,llm_client: HelloAgenticsLLM, tool_executor: ToolExecutor, max_steps: int = 5):
        self.llm_client = llm_client
        self.tool_executor = tool_executor
        self.max_steps = max_steps
        self.history = []
    
    def _parse_output(self, text: str):
        thought_match = re.search(r"Thought:(.*)", text)
        action_match = re.search(r"Action:(.*)", text)

        thought = thought_match.group(1).strip() if thought_match else None
        action_raw = action_match.group(1).strip() if action_match else None

        if not action_raw:
            return thought, None

        # 🔥 只取 Action 行中第一个 协议形式
        m = re.search(r"(Finish\[[^\]]+\]|\w+\[[^\]]+\])", action_raw)
        action = m.group(1) if m else None

        return thought, action

    
    def _parse_action(self, action_text: str):
        match = re.match(r"(\w+)\[(.*)\]", action_text)
        if match:
            return match.group(1), match.group(2)
        return None, None

    def run(self, question: str):
        self.history = []
        current_step = 0

        while current_step < self.max_steps:
            current_step += 1
            print(f"--- 第 {current_step} 步 ---")

            tools_desc = self.tool_executor.getAvaliableTools()
            history_str = '\n'.join(self.history)
            prompt = REACT_PROMPT_TEMPLATE.format(
                tools = tools_desc,
                question=question,
                history=history_str
            )

            messages = [{"role": "user", "content": prompt}]
            response_text = self.llm_client.think(messages=messages)

            if not response_text:
                print("错误：LLM未能返回有效响应。")
                break
                
            though, action = self._parse_output(response_text)

            if though:
                print(f"思考：{though}")
            
            if not action:
                print("警告：未能解析出有效的Action，流程终止。")
                break

            # if action.startswith("Finish"):
            #     final_answer = re.match(r"Finish\[(.*)\]", action).group(1)
            #     print(f"🎉最终答案：{final_answer}")
            #     return final_answer
            if "Finish[" in action:
                final_answer = re.search(r"Finish\[(.*)\]", action).group(1)
                print(f"🎉 最终答案：{final_answer}")
                return final_answer
            
            tool_name, tool_input = self._parse_action(action)
            if not tool_name or not tool_input:
                print(f"❌ 非法 Action，终止：{action}")
                break

            print(f"🎬行动：{tool_name}[{tool_input}]")

            tool_function = self.tool_executor.getTool(tool_name)
            if not tool_function:
                observation = f"错误：未找到名为‘{tool_name}' 的工具。"
            else:
                observation = tool_function(tool_input)

            print(f"👀观察： {observation}")

            self.history.append(f"Action: {action}")
            self.history.append(f"Observation: {observation}")

            import time
            time.sleep(0.8)

        print("已经到最大部署，流程终止")
        return None

In [53]:
llm_client = HelloAgenticsLLM()
tool_executor = ToolExecutor()
search_description = "一个网页搜索引擎。当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
tool_executor.registerTool(name="Search", description=search_description, func=search)
question = "最新款的Macbook pro什么时候发布，大概会有什吗提升?"
react = ReActAgent(llm_client, tool_executor, max_steps=20)


react.run(question=question)

工具 Search 已注册。
--- 第 1 步 ---
🧠正字调用 mimo-v2-flash-free 模型...
✅ 大模型响应成功：
Thought: 用户询问最新款MacBook Pro的发布时间和可能的提升。这是一个关于科技产品最新动态的问题，我的知识库可能不是最新的，因此需要使用Search工具来获取最新的信息。

Action: 我将使用Search工具来查找关于最新款MacBook Pro的发布日期和可能的提升信息。

Search[最新款MacBook Pro发布日期和提升]Thought: 用户询问最新款MacBook Pro的发布时间和可能的提升。这是一个关于科技产品最新动态的问题，我的知识库可能不是最新的，因此需要使用Search工具来获取最新的信息。

Action: 我将使用Search工具来查找关于最新款MacBook Pro的发布日期和可能的提升信息。

Search[最新款MacBook Pro发布日期和提升]思考：用户询问最新款MacBook Pro的发布时间和可能的提升。这是一个关于科技产品最新动态的问题，我的知识库可能不是最新的，因此需要使用Search工具来获取最新的信息。
警告：未能解析出有效的Action，流程终止。
已经到最大部署，流程终止
